In [1]:
import os
import pandas as pd

from src.data_preprocessing.helpers import get_video_info

In [2]:
path = os.path.expandvars("$SCRATCH/gesture/data/wlaslvideos")

In [3]:
videos_info = []
corrupted_files = []

for file_path in os.listdir(path):
    info = get_video_info(os.path.join(path, file_path))
    if info is None:
        corrupted_files.append(file_path)
        continue

    info["filename"] = file_path
    
    videos_info.append(info)

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x556a8cbb4b80] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x556a8cbb4b80] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x556a8cd3b0c0] moov atom not found


In [4]:
df_video_info = pd.DataFrame(videos_info)
df_video_info["video_id"] = df_video_info["filename"].str.split(".").str[0]

In [5]:
df_video_info.head()

,frame_count,fps,duration_seconds,width,height,filename,video_id
0,113,30.002655,3.766333,640,480,37959.mp4,37959
1,43,23.976061,1.793456,1920,1080,22954.mp4,22954
2,30,29.970000,1.001001,288,192,21951.mp4,21951
3,53,25.000000,2.120000,480,320,45927.mp4,45927
4,100,25.000000,4.000000,626,360,70342.mp4,70342


In [6]:
df_video_info.count()

frame_count         13185
fps                 13185
duration_seconds    13185
width               13185
height              13185
filename            13185
video_id            13185
dtype: int64

In [7]:
df_wlasl_info = pd.read_json("data/WLASL_v0.3.json")  # from https://www.kaggle.com/datasets/risangbaskoro/wlasl-processed?resource=download&select=WLASL_v0.3.json
df_exploded = df_wlasl_info.explode("instances")
df_wlasl_info = pd.concat(
    [df_exploded["gloss"], df_exploded["instances"].apply(pd.Series)], axis=1
)

In [ ]:
df_wlasl_info["gloss"].value_counts().describe()

count    2000.000000
mean       10.541500
std         3.547192
min         6.000000
25%         8.000000
50%        10.000000
75%        12.000000
max        40.000000
Name: count, dtype: float64

In [9]:
df_wlasl_info.count()

gloss           21083
bbox            21083
fps             21083
frame_end       21083
frame_start     21083
instance_id     21083
signer_id       21083
source          21083
split           21083
url             21083
variation_id    21083
video_id        21083
dtype: int64

In [ ]:
intersection = pd.merge(df_video_info, df_wlasl_info, how='inner', left_on=["video_id"], right_on=["video_id"])
df_video_info = intersection

In [ ]:
df_gloss_map = pd.read_csv("data/gloss_map.csv")

In [ ]:
df_100 = pd.merge(
    df_video_info,
    df_gloss_map,
    how="inner",
    left_on=["gloss"],
    right_on=["gloss"],
)

In [13]:
df_100.head()

,frame_count,fps_x,duration_seconds,width,height,filename,video_id,gloss,bbox,fps_y,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,label
0,30,29.97000,1.001001,288,192,21951.mp4,21951,finish,"[96, 16, 231, 192]",25,-1,1,18,11,signingsavvy,train,https://www.signingsavvy.com/signs/mp4/8/8336.mp4,1,180
1,100,25.00000,4.000000,626,360,70342.mp4,70342,friend,"[188, 38, 449, 360]",25,3060,2961,2,121,northtexas,train,https://www.youtube.com/watch?v=bq-HmgjGzmw,0,183
2,50,25.00000,2.000000,480,320,28734.mp4,28734,if,"[0, 0, 360, 240]",25,233,184,0,9,asl5200,train,https://www.youtube.com/watch?v=K1YuKDdDp_k,0,205
3,77,29.97003,2.569233,720,480,62253.mp4,62253,want,"[93, 19, 424, 360]",25,-1,1,11,5,aslu,train,https://www.youtube.com/watch?v=aGzNpjV6-0I,0,98
4,73,23.97609,3.044700,656,370,65365.mp4,65365,clean,"[159, 17, 466, 370]",25,-1,1,3,90,aslsignbank,val,https://aslsignbank.haskins.yale.edu/dictionar...,0,152


In [ ]:
df_100["gloss"].count()

1883

In [15]:
df_100["gloss"].nunique()

208